In [1]:
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import Latest
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage, AverageDollarVolume, Returns, RSI
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.filters import Q500US, Q1500US
from quantopian.pipeline.data.quandl import fred_usdontd156n as libor
from quantopian.pipeline.data.zacks import EarningsSurprises

import talib
import pandas as pd
import numpy as np
from time import time

import alphalens as al
import pyfolio as pf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition, ensemble, preprocessing, isotonic, metrics

In [2]:
bs = morningstar.balance_sheet
cfs = morningstar.cash_flow_statement
is_ = morningstar.income_statement
or_ = morningstar.operation_ratios
er = morningstar.earnings_report
v = morningstar.valuation
vr = morningstar.valuation_ratios


def make_factors():
    def Asset_Growth_3M():
        return Returns(inputs=[bs.total_assets], window_length=63)

    def Asset_To_Equity_Ratio():
        return bs.total_assets.latest / bs.common_stock_equity.latest

    def Capex_To_Cashflows():
        return (cfs.capital_expenditure.latest * 4.) / \
            (cfs.free_cash_flow.latest * 4.)
        
    def EBITDA_Yield():
        return (is_.ebitda.latest * 4.) / \
            USEquityPricing.close.latest        

    def EBIT_To_Assets():
        return (is_.ebit.latest * 4.) / \
            bs.total_assets.latest
        
    def Earnings_Quality():
        return morningstar.cash_flow_statement.operating_cash_flow.latest / \
               EarningsSurprises.eps_act.latest
        
    def Return_On_Total_Invest_Capital():
        return or_.roic.latest
    
    class Mean_Reversion_1M(CustomFactor):
        inputs = [Returns(window_length=21)]
        window_length = 252

        def compute(self, today, assets, out, monthly_rets):
            out[:] = (monthly_rets[-1] - np.nanmean(monthly_rets, axis=0)) / \
                np.nanstd(monthly_rets, axis=0)
                
    class MACD_Signal_10d(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 60

        def compute(self, today, assets, out, close):

            sig_lines = []

            for col in close.T:
                # get signal line only
                try:
                    _, signal_line, _ = talib.MACD(col, fastperiod=12,
                                                   slowperiod=26, signalperiod=10)
                    sig_lines.append(signal_line[-1])
                # if error calculating, return NaN
                except:
                    sig_lines.append(np.nan)
            out[:] = sig_lines 
            
    class Moneyflow_Volume_5d(CustomFactor):
        inputs = [USEquityPricing.close, USEquityPricing.volume]
        window_length = 5

        def compute(self, today, assets, out, close, volume):

            mfvs = []

            for col_c, col_v in zip(close.T, volume.T):

                # denominator
                denominator = np.dot(col_c, col_v)

                # numerator
                numerator = 0.
                for n, price in enumerate(col_c.tolist()):
                    if price > col_c[n - 1]:
                        numerator += price * col_v[n]
                    else:
                        numerator -= price * col_v[n]

                mfvs.append(numerator / denominator)
            out[:] = mfvs  
            
           
    def Net_Income_Margin():
        return or_.net_margin.latest           

    def Operating_Cashflows_To_Assets():
        return (cfs.operating_cash_flow.latest * 4.) / \
            bs.total_assets.latest

    def Price_Momentum_3M():
        return Returns(window_length=63)
    
    class Price_Oscillator(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 252

        def compute(self, today, assets, out, close):
            four_week_period = close[-20:]
            out[:] = (np.nanmean(four_week_period, axis=0) /
                      np.nanmean(close, axis=0)) - 1.
    
    def Returns_39W():
        return Returns(window_length=215)
    
    class Trendline(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 252

        # using MLE for speed
        def compute(self, today, assets, out, close):

            # prepare X matrix (x_is - x_bar)
            X = range(self.window_length)
            X_bar = np.nanmean(X)
            X_vector = X - X_bar
            X_matrix = np.tile(X_vector, (len(close.T), 1)).T

            # prepare Y matrix (y_is - y_bar)
            Y_bar = np.nanmean(close, axis=0)
            Y_bars = np.tile(Y_bar, (self.window_length, 1))
            Y_matrix = close - Y_bars

            # prepare variance of X
            X_var = np.nanvar(X)

            # multiply X matrix an Y matrix and sum (dot product)
            # then divide by variance of X
            # this gives the MLE of Beta
            out[:] = (np.sum((X_matrix * Y_matrix), axis=0) / X_var) / \
                (self.window_length)
        
    class Vol_3M(CustomFactor):
        inputs = [Returns(window_length=2)]
        window_length = 63

        def compute(self, today, assets, out, rets):
            out[:] = np.nanstd(rets, axis=0)
            
    def Working_Capital_To_Assets():
        return bs.working_capital.latest / bs.total_assets.latest
        
    all_factors = {
        'Asset Growth 3M': Asset_Growth_3M,
        'Asset to Equity Ratio': Asset_To_Equity_Ratio,
        'Capex to Cashflows': Capex_To_Cashflows,
        'EBIT to Assets': EBIT_To_Assets,
        'EBITDA Yield': EBITDA_Yield,        
        'Earnings Quality': Earnings_Quality,
        'MACD Signal Line': MACD_Signal_10d,
        'Mean Reversion 1M': Mean_Reversion_1M,
        'Moneyflow Volume 5D': Moneyflow_Volume_5d,
        'Net Income Margin': Net_Income_Margin,        
        'Operating Cashflows to Assets': Operating_Cashflows_To_Assets,
        'Price Momentum 3M': Price_Momentum_3M,
        'Price Oscillator': Price_Oscillator,
        'Return on Invest Capital': Return_On_Total_Invest_Capital,
        '39 Week Returns': Returns_39W,
        'Trendline': Trendline,
        'Vol 3M': Vol_3M,
        'Working Capital to Assets': Working_Capital_To_Assets,        
    }        
    
    return all_factors

<a></a></a>

In [3]:
universe = Q1500US()

factors = make_factors()

In [4]:
n_fwd_days = 5 # number of days to compute returns over

In [5]:
def make_history_pipeline(factors, universe, n_fwd_days=5):
    # Call .rank() on all factors and mask out the universe
    factor_ranks = {name: f().rank(mask=universe) for name, f in factors.iteritems()}
    # Get cumulative returns over last n_fwd_days days. We will later shift these.
    factor_ranks['Returns'] = Returns(inputs=[USEquityPricing.open],
                                      mask=universe, window_length=n_fwd_days)
    
    pipe = Pipeline(screen=universe, columns=factor_ranks)
    
    return pipe

In [6]:
history_pipe = make_history_pipeline(factors, universe, n_fwd_days=n_fwd_days)

In [7]:
start_timer = time()
start = pd.Timestamp("2016-03-06")
end = pd.Timestamp("2016-09-14")
results = run_pipeline(history_pipe, start_date=start, end_date=end)
results.index.names = ['date', 'security']
end_timer = time()

/usr/local/lib/python2.7/dist-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/numpy/lib/nanfunctions.py:1202: RuntimeWarning: Degrees of freedom <= 0 for slice.
  warnings.warn("Degrees of freedom <= 0 for slice.", RuntimeWarning)


In [8]:
print "Time to run pipeline %.2f secs" % (end_timer - start_timer)

Time to run pipeline 70.10 secs


In [9]:
results.head()

39 Week Returns  Asset Growth 3M  \
date                      security                                              
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])             289.0            572.0   
                          Equity(24 [AAPL])            489.0           1040.0   
                          Equity(53 [ABMD])           1403.0           1313.0   
                          Equity(62 [ABT])             548.0            436.0   
                          Equity(67 [ADSK])            800.0            595.0   

                                             Asset to Equity Ratio  \
date                      security                                   
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])                   927.0   
                          Equity(24 [AAPL])                  619.0   
                          Equity(53 [ABMD])                   93.0   
                          Equity(62 [ABT])                   467.0   
                          Equity(67 [ADSK])                  918.0   

                                             Capex to Cashflows  \
date                      security                                
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])                243.0   
                          Equity(24 [AAPL])               691.0   
                          Equity(53 [ABMD])               627.0   
                          Equity(62 [ABT])                410.0   
                          Equity(67 [ADSK])               681.0   

                                             EBIT to Assets  EBITDA Yield  \
date                      security                                          
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])            172.0          23.0   
                          Equity(24 [AAPL])          1373.0        1489.0   
                          Equity(53 [ABMD])          1222.0         236.0   
                          Equity(62 [ABT])            837.0        1416.0   
                          Equity(67 [ADSK])           248.0         269.0   

                                             Earnings Quality  \
date                      security                              
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])                NaN   
                          Equity(24 [AAPL])            1453.0   
                          Equity(53 [ABMD])             575.0   
                          Equity(62 [ABT])             1328.0   
                          Equity(67 [ADSK])              31.0   

                                             MACD Signal Line  \
date                      security                              
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])              882.0   
                          Equity(24 [AAPL])             307.0   
                          Equity(53 [ABMD])             266.0   
                          Equity(62 [ABT])              345.0   
                          Equity(67 [ADSK])             548.0   

                                             Mean Reversion 1M  \
date                      security                               
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])              1051.0   
                          Equity(24 [AAPL])             1025.0   
                          Equity(53 [ABMD])               66.0   
                          Equity(62 [ABT])               531.0   
                          Equity(67 [ADSK])              911.0   

                                             Moneyflow Volume 5D  \
date                      security                                 
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])                 941.0   
                          Equity(24 [AAPL])               1393.0   
                          Equity(53 [ABMD])                711.0   
                          Equity(62 [ABT])                 460.0   
                          Equity(67 [ADSK])               1088.0   

                                             Net Income Margin  \
date                      security                        

In [10]:
results.tail()

39 Week Returns  \
date                      security                                 
2016-09-14 00:00:00+00:00 Equity(49666 [AGR])                NaN   
                          Equity(49700 [FCE_A])              NaN   
                          Equity(49925 [NGVT])               NaN   
                          Equity(50049 [FTV])                NaN   
                          Equity(50077 [TWLO])               NaN   

                                                 Asset Growth 3M  \
date                      security                                 
2016-09-14 00:00:00+00:00 Equity(49666 [AGR])              448.0   
                          Equity(49700 [FCE_A])            594.0   
                          Equity(49925 [NGVT])            1414.0   
                          Equity(50049 [FTV])             1347.0   
                          Equity(50077 [TWLO])            1480.0   

                                                 Asset to Equity Ratio  \
date                      security                                       
2016-09-14 00:00:00+00:00 Equity(49666 [AGR])                    484.0   
                          Equity(49700 [FCE_A])                  990.0   
                          Equity(49925 [NGVT])                  1295.0   
                          Equity(50049 [FTV])                    975.0   
                          Equity(50077 [TWLO])                   136.0   

                                                 Capex to Cashflows  \
date                      security                                    
2016-09-14 00:00:00+00:00 Equity(49666 [AGR])                  71.0   
                          Equity(49700 [FCE_A])              1323.0   
                          Equity(49925 [NGVT])                481.0   
                          Equity(50049 [FTV])                 830.0   
                          Equity(50077 [TWLO])               1238.0   

                                                 EBIT to Assets  EBITDA Yield  \
date                      security                                              
2016-09-14 00:00:00+00:00 Equity(49666 [AGR])             490.0        1274.0   
                          Equity(49700 [FCE_A])            94.0          20.0   
                          Equity(49925 [NGVT])           1254.0         466.0   
                          Equity(50049 [FTV])            1164.0        1107.0   
                          Equity(50077 [TWLO])            107.0         149.0   

                                                 Earnings Quality  \
date                      security                                  
2016-09-14 00:00:00+00:00 Equity(49666 [AGR])              1353.0   
                          Equity(49700 [FCE_A])            1207.0   
                          Equity(49925 [NGVT])              412.0   
                          Equity(50049 [FTV])              1129.0   
                          Equity(50077 [TWLO])              226.0   

                                                 MACD Signal Line  \
date                      security                                  
2016-09-14 00:00:00+00:00 Equity(49666 [AGR])               184.0   
                          Equity(49700 [FCE_A])             705.0   
                          Equity(49925 [NGVT])             1269.0   
                          Equity(50049 [FTV])              1026.0   
                          Equity(50077 [TWLO])             1465.0   

                                                 Mean Reversion 1M  \
date                      security                                   
2016-09-14 00:00:00+00:00 Equity(49666 [AGR])                268.0   
                          Equity(49700 [FCE_A])              469.0   
                          Equity(49925 [NGVT])                30.0   
                          Equity(50049 [FTV])                 11.0   
                          Equity(50077 [TWLO])                22.0   

                                                 Mo

In [11]:
def shift_mask_data(X, Y, upper_percentile=70, lower_percentile=30, n_fwd_days=1):
    # Shift X to match factors at t to returns at t+n_fwd_days (we want to predict future returns after all)
    shifted_X = np.roll(X, n_fwd_days+1, axis=0)
    
    # Slice off rolled elements
    X = shifted_X[n_fwd_days+1:]
    Y = Y[n_fwd_days+1:]
    
    n_time, n_stocks, n_factors = X.shape
    
    # Look for biggest up and down movers
    # READ ON WHATS HAPPENING WITH THIS np.newaxis thing
    
    upper = np.nanpercentile(Y, upper_percentile, axis=1)[:, np.newaxis]
    lower = np.nanpercentile(Y, lower_percentile, axis=1)[:, np.newaxis]
  
    upper_mask = (Y >= upper)
    lower_mask = (Y <= lower)
    
    mask = upper_mask | lower_mask # This also drops nans
    mask = mask.flatten()
    
    # Only try to predict whether a stock moved up/down relative to other stocks
    Y_binary = np.zeros(n_time * n_stocks)
    Y_binary[upper_mask.flatten()] = 1
    Y_binary[lower_mask.flatten()] = -1
    
    # Flatten X
    X = X.reshape((n_time * n_stocks, n_factors))

    # Drop stocks that did not move much (i.e. are in the 30th to 70th percentile)
    X = X[mask]
    Y_binary = Y_binary[mask]
    
    return X, Y_binary

In [12]:
results

39 Week Returns  \
date                      security                                 
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])                 289.0   
                          Equity(24 [AAPL])                489.0   
                          Equity(53 [ABMD])               1403.0   
                          Equity(62 [ABT])                 548.0   
                          Equity(67 [ADSK])                800.0   
                          Equity(76 [TAP])                1380.0   
                          Equity(114 [ADBE])              1217.0   
                          Equity(122 [ADI])                717.0   
                          Equity(128 [ADM])                442.0   
                          Equity(161 [AEP])               1211.0   
                          Equity(166 [AES])                519.0   
                          Equity(168 [AET])                960.0   
                          Equity(185 [AFL])                886.0   
                          Equity(197 [AGCO])               993.0   
                          Equity(216 [HES])                192.0   
                          Equity(239 [AIG])                782.0   
                          Equity(266 [AJG])                616.0   
                          Equity(270 [AKRX])               139.0   
                          Equity(300 [ALK])               1338.0   
                          Equity(301 [ALKS])               131.0   
                          Equity(337 [AMAT])               914.0   
                          Equity(351 [AMD])               1019.0   
                          Equity(353 [AME])                741.0   
                          Equity(357 [TWX])                502.0   
                          Equity(368 [AMGN])               752.0   
                          Equity(410 [AN])                 556.0   
                          Equity(438 [AON])                985.0   
                          Equity(448 [APA])                291.0   
                          Equity(455 [APC])                 73.0   
                          Equity(460 [APD])                684.0   
...                                                          ...   
2016-09-14 00:00:00+00:00 Equity(49139 [FIT])               12.0   
                          Equity(49176 [TRU])             1280.0   
                          Equity(49183 [WRK])              737.0   
                          Equity(49203 [GCI])              129.0   
                          Equity(49204 [CABO])            1264.0   
                          Equity(49210 [CC])              1451.0   
                          Equity(49213 [ENR])             1007.0   
                          Equity(49229 [KHC])             1215.0   
                          Equity(49238 [SRG])             1147.0   
                          Equity(49242 [PYPL])             882.0   
                          Equity(49271 [OLLI])            1421.0   
                          Equity(49279 [BUFF])            1338.0   
                          Equity(49288 [LITE])            1476.0   
                          Equity(49325 [CCP])              540.0   
                          Equity(49413 [PEN])             1464.0   
                          Equity(49458 [MSG])              677.0   
                          Equity(49464 [PSTG])             143.0   
                          Equity(49496 [FDC])              284.0   
                          Equity(49506 [HPE])             1440.0   
                          Equity(49515 [RACE])             427.0   
                          Equity(49543 [FCPT])            1477.0   
                          Equity(49608 [MTCH])               NaN   
                          Equity(49610 [SQ])                 NaN   
                          Equity(49630 [CSRA])               NaN   
                          Equity(49655 [TEAM])               NaN   
                          Equity(49666 [AGR])                NaN   
                          Equity(4970

In [13]:
#Importand swapping of axes done here, reasoning not sure though

In [14]:
# Massage data to be in the form expected by shift_mask_data()
results_wo_returns = results.copy()
returns = results_wo_returns.pop('Returns')
Y = returns.unstack().values
X = results_wo_returns.to_panel() 
X = X.swapaxes(2, 0).swapaxes(0, 1).values # (factors, time, stocks) -> (time, stocks, factors)

In [68]:
type(X)

<type 'numpy.ndarray'>

In [69]:
X.shape

(134, 1615, 18)

In [70]:
returns.shape

(200195,)

In [57]:
Xtop = X[0:5]

In [58]:
Xtop

array([[[  289.,   572.,   927., ...,   695.,  1304.,   550.],
        [  489.,  1040.,   619., ...,   144.,   536.,   253.],
        [ 1403.,  1313.,    93., ...,  1408.,  1244.,  1273.],
        ..., 
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan]],

       [[  336.,   605.,   924., ...,   695.,  1302.,   551.],
        [  478.,   991.,   618., ...,   143.,   545.,   258.],
        [ 1426.,  1288.,    93., ...,  1408.,  1220.,  1273.],
        ..., 
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan]],

       [[  249.,   610.,   923., ...,   694.,  1314.,   551.],
        [  469.,   990.,   618., ...,   144.,   514.,   258.],
        [ 1421.,  1287.,    93., ...,  1407.,  1220.,  1272.],
        ..., 
        [

In [59]:
Xrolled1 = np.roll(Xtop,shift=1,axis=0)

In [60]:
Xrolled1

array([[[  281.,   638.,   924., ...,   708.,  1301.,   553.],
        [  499.,   973.,   618., ...,   141.,   520.,   260.],
        [ 1219.,  1279.,    92., ...,  1406.,  1186.,  1272.],
        ..., 
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan]],

       [[  289.,   572.,   927., ...,   695.,  1304.,   550.],
        [  489.,  1040.,   619., ...,   144.,   536.,   253.],
        [ 1403.,  1313.,    93., ...,  1408.,  1244.,  1273.],
        ..., 
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan]],

       [[  336.,   605.,   924., ...,   695.,  1302.,   551.],
        [  478.,   991.,   618., ...,   143.,   545.,   258.],
        [ 1426.,  1288.,    93., ...,  1408.,  1220.,  1273.],
        ..., 
        [

In [22]:
results.shape

(200195, 19)

In [23]:
results.head()

39 Week Returns  Asset Growth 3M  \
date                      security                                              
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])             289.0            572.0   
                          Equity(24 [AAPL])            489.0           1040.0   
                          Equity(53 [ABMD])           1403.0           1313.0   
                          Equity(62 [ABT])             548.0            436.0   
                          Equity(67 [ADSK])            800.0            595.0   

                                             Asset to Equity Ratio  \
date                      security                                   
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])                   927.0   
                          Equity(24 [AAPL])                  619.0   
                          Equity(53 [ABMD])                   93.0   
                          Equity(62 [ABT])                   467.0   
                          Equity(67 [ADSK])                  918.0   

                                             Capex to Cashflows  \
date                      security                                
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])                243.0   
                          Equity(24 [AAPL])               691.0   
                          Equity(53 [ABMD])               627.0   
                          Equity(62 [ABT])                410.0   
                          Equity(67 [ADSK])               681.0   

                                             EBIT to Assets  EBITDA Yield  \
date                      security                                          
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])            172.0          23.0   
                          Equity(24 [AAPL])          1373.0        1489.0   
                          Equity(53 [ABMD])          1222.0         236.0   
                          Equity(62 [ABT])            837.0        1416.0   
                          Equity(67 [ADSK])           248.0         269.0   

                                             Earnings Quality  \
date                      security                              
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])                NaN   
                          Equity(24 [AAPL])            1453.0   
                          Equity(53 [ABMD])             575.0   
                          Equity(62 [ABT])             1328.0   
                          Equity(67 [ADSK])              31.0   

                                             MACD Signal Line  \
date                      security                              
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])              882.0   
                          Equity(24 [AAPL])             307.0   
                          Equity(53 [ABMD])             266.0   
                          Equity(62 [ABT])              345.0   
                          Equity(67 [ADSK])             548.0   

                                             Mean Reversion 1M  \
date                      security                               
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])              1051.0   
                          Equity(24 [AAPL])             1025.0   
                          Equity(53 [ABMD])               66.0   
                          Equity(62 [ABT])               531.0   
                          Equity(67 [ADSK])              911.0   

                                             Moneyflow Volume 5D  \
date                      security                                 
2016-03-07 00:00:00+00:00 Equity(2 [ARNC])                 941.0   
                          Equity(24 [AAPL])               1393.0   
                          Equity(53 [ABMD])                711.0   
                          Equity(62 [ABT])                 460.0   
                          Equity(67 [ADSK])               1088.0   

                                             Net Income Margin  \
date                      security                        

In [24]:
returns

date                       security             
2016-03-07 00:00:00+00:00  Equity(2 [ARNC])         0.071508
                           Equity(24 [AAPL])        0.056886
                           Equity(53 [ABMD])        0.028731
                           Equity(62 [ABT])        -0.012456
                           Equity(67 [ADSK])        0.083758
                           Equity(76 [TAP])         0.050067
                           Equity(114 [ADBE])       0.022323
                           Equity(122 [ADI])        0.041339
                           Equity(128 [ADM])        0.060650
                           Equity(161 [AEP])       -0.000163
                           Equity(166 [AES])        0.058883
                           Equity(168 [AET])       -0.003470
                           Equity(185 [AFL])        0.017448
                           Equity(197 [AGCO])       0.064570
                           Equity(216 [HES])        0.142160
                           Equity(23

In [25]:
Y

array([[ 0.07150838,  0.05688623,  0.02873134, ...,         nan,
                nan,         nan],
       [ 0.05921053,  0.04854071,  0.04136223, ...,         nan,
                nan,         nan],
       [ 0.06775956,  0.0026863 ,  0.00843731, ...,         nan,
                nan,         nan],
       ..., 
       [-0.01470588, -0.02841226,  0.00940678, ..., -0.01436266,
        -0.02357601,  0.03768433],
       [-0.06483301, -0.04865616, -0.01357234, ..., -0.03158602,
        -0.06504673, -0.05425824],
       [-0.06504854, -0.00296763,  0.00346313, ..., -0.05856451,
        -0.02994469, -0.07342193]])

In [61]:
Y.shape

(134, 1615)

In [67]:
X.shape

(134, 1615, 18)

In [26]:
Y[-1]

array([-0.06504854, -0.00296763,  0.00346313, ..., -0.05856451,
       -0.02994469, -0.07342193])

In [27]:
Y.shape

(134, 1615)

In [28]:
type(Y)

<type 'numpy.ndarray'>

In [29]:
returns[-1]

-0.073421926910299029

In [30]:
results.tail(1)

,,39 Week Returns,Asset Growth 3M,Asset to Equity Ratio,Capex to Cashflows,EBIT to Assets,EBITDA Yield,Earnings Quality,MACD Signal Line,Mean Reversion 1M,Moneyflow Volume 5D,Net Income Margin,Operating Cashflows to Assets,Price Momentum 3M,Price Oscillator,Return on Invest Capital,Returns,Trendline,Vol 3M,Working Capital to Assets
date,security,,,,,,,,,,,,,,,,,,,
2016-09-14 00:00:00+00:00,Equity(50077 [TWLO]),NaN,1480.0,136.0,1238.0,107.0,149.0,226.0,1465.0,22.0,1256.0,142.0,151.0,NaN,1312.0,130.0,-0.073422,NaN,1481.0,1305.0


In [31]:
results.tail(2)

39 Week Returns  \
date                      security                                
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])               NaN   
                          Equity(50077 [TWLO])              NaN   

                                                Asset Growth 3M  \
date                      security                                
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])            1347.0   
                          Equity(50077 [TWLO])           1480.0   

                                                Asset to Equity Ratio  \
date                      security                                      
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])                   975.0   
                          Equity(50077 [TWLO])                  136.0   

                                                Capex to Cashflows  \
date                      security                                   
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])                830.0   
                          Equity(50077 [TWLO])              1238.0   

                                                EBIT to Assets  EBITDA Yield  \
date                      security                                             
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])           1164.0        1107.0   
                          Equity(50077 [TWLO])           107.0         149.0   

                                                Earnings Quality  \
date                      security                                 
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])             1129.0   
                          Equity(50077 [TWLO])             226.0   

                                                MACD Signal Line  \
date                      security                                 
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])             1026.0   
                          Equity(50077 [TWLO])            1465.0   

                                                Mean Reversion 1M  \
date                      security                                  
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])                11.0   
                          Equity(50077 [TWLO])               22.0   

                                                Moneyflow Volume 5D  \
date                      security                                    
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])                 816.0   
                          Equity(50077 [TWLO])               1256.0   

                                                Net Income Margin  \
date                      security                                  
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])              1146.0   
                          Equity(50077 [TWLO])              142.0   

                                                Operating Cashflows to Assets  \
date                      security                                              
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])                          1188.0   
                          Equity(50077 [TWLO])                          151.0   

                                                Price Momentum 3M  \
date                      security                                  
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])               921.0   
                          Equity(50077 [TWLO])                NaN   

                                                Price Oscillator  \
date                      security                                 
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])              506.0   
                          Equity(50077 [TWLO])            1312.0   

                                                Return on Invest Capital  \
date                      security                                         
2016-09-14 00:00:00+00:00 Equity(50049 [FTV])                     1141.0   
                          Equity(50077 [TWLO])                     130.0   

                                                 R

In [32]:
temp = results_wo_returns.to_panel()

In [33]:
temp =  temp.swapaxes(2, 0).swapaxes(0, 1)

In [34]:
temp

<class 'pandas.core.panel.Panel'>
Dimensions: 134 (items) x 1615 (major_axis) x 18 (minor_axis)
Items axis: 2016-03-07 00:00:00+00:00 to 2016-09-14 00:00:00+00:00
Major_axis axis: Equity(2 [ARNC]) to Equity(50077 [TWLO])
Minor_axis axis: 39 Week Returns to Working Capital to Assets

In [35]:
results_wo_returns.index = results_wo_returns.index.set_levels(results_wo_returns.index.get_level_values(1).map(lambda x: x.symbol), 1, )


In [36]:
results_wo_returns.index = results_wo_returns.index.set_levels(results_wo_returns.index.get_level_values(0).map(lambda x: x.date), 0, )

In [37]:
results_wo_returns.sample(10).sort()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: sort(....) is deprecated, use sort_index(.....)
  """Entry point for launching an IPython kernel.


39 Week Returns  Asset Growth 3M  Asset to Equity Ratio  \
date       security                                                            
2016-03-07 ATHN                 18.0            989.0                  897.0   
           GT                  699.0             24.0                 1186.0   
           JEC                 696.0            257.0                  295.0   
           KN                  106.0           1146.0                  937.0   
           SCCO               1182.0            672.0                  177.0   
           SRE                 511.0            818.0                  808.0   
           STZ                1174.0            821.0                   50.0   
           TDS                1105.0            470.0                  783.0   
           TXRH                773.0            936.0                 1323.0   
           VA                  941.0             87.0                   82.0   

                     Capex to Cashflows  EBIT to Assets  EBITDA Yield  \
date       security                                                     
2016-03-07 ATHN                   445.0           185.0         213.0   
           GT                    1179.0           640.0        1484.0   
           JEC                    724.0           939.0         675.0   
           KN                     812.0           767.0        1118.0   
           SCCO                   905.0          1200.0         362.0   
           SRE                    768.0           914.0        1292.0   
           STZ                    618.0          1058.0        1062.0   
           TDS                    312.0           795.0         796.0   
           TXRH                     NaN           380.0        1220.0   
           VA                    1118.0            55.0         121.0   

                     Earnings Quality  MACD Signal Line  Mean Reversion 1M  \
date       security                                                          
2016-03-07 ATHN                 191.0             234.0              797.0   
           GT                    12.0             455.0              553.0   
           JEC                  662.0             415.0              121.0   
           KN                   866.0             154.0              363.0   
           SCCO                 399.0            1416.0             1070.0   
           SRE                 1088.0              93.0              705.0   
           STZ                 1249.0             877.0              732.0   
           TDS                  795.0            1099.0              255.0   
           TXRH                1123.0             300.0               41.0   
           VA                   282.0             112.0              335.0   

                     Moneyflow Volume 5D  Net Income Margin  \
date       security                                           
2016-03-07 ATHN                    292.0              276.0   
           GT                      359.0              825.0   
           JEC                    1126.0             1457.0   
           KN                      790.0              650.0   
           SCCO                    583.0             1214.0   
           SRE                    1079.0              546.0   
           STZ                     249.0              654.0   
           TDS                     678.0              590.0   
           TXRH                    243.0             1120.0   
           VA                      209.0               24.0   

                     Operating Cashflows to Assets  Price Momentum 3M  \
date       security                                                     
2016-03-07 ATHN                              964.0               46.0   
           GT                                127.0              758.0   
           JEC                              1124.0              483.0   
           KN                                808.0               77.0   
           SCCO                             1255.0 

In [38]:
tmp = (returns > 0.).to_frame()
tmp.index = tmp.index.set_levels(tmp.index.get_level_values(1).map(lambda x: x.symbol), 1)
tmp.columns = ['5-day forward returns > 0']
tmp.sample(10).sort()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: sort(....) is deprecated, use sort_index(.....)
  after removing the cwd from sys.path.


,,5-day forward returns > 0
date,security,
2016-03-09 00:00:00+00:00,DLX,True
2016-04-04 00:00:00+00:00,LOW,False
2016-04-06 00:00:00+00:00,MDU,True
2016-04-22 00:00:00+00:00,ABT,True
2016-04-26 00:00:00+00:00,EPAM,False
2016-05-24 00:00:00+00:00,ADM,False
2016-06-30 00:00:00+00:00,AIZ,True
2016-07-29 00:00:00+00:00,BRO,True
2016-08-01 00:00:00+00:00,JAH,False


In [39]:
# Train-test split
train_size_perc = 0.8
n_time, n_stocks, n_factors = X.shape
train_size = np.int16(np.round(train_size_perc * n_time))
X_train, Y_train = X[:train_size, ...], Y[:train_size]

# NOTE THE INDEX BELOW, THAT'S BECAUSE n_fwd_days will be sliced off when X_train, Y_train are passed through shift_mask_data
X_test, Y_test = X[(train_size+n_fwd_days):, ...], Y[(train_size+n_fwd_days):]

In [40]:
X_train_shift, Y_train_shift = shift_mask_data(X_train, Y_train, n_fwd_days=n_fwd_days)
X_test_shift, Y_test_shift = shift_mask_data(X_test, Y_test, n_fwd_days=n_fwd_days, 
                                             lower_percentile=50, 
                                             upper_percentile=50)

In [41]:
X_train_shift.shape

(90584, 18)

In [42]:
X_train_shift.shape, X_test_shift.shape

((90584, 18), (23894, 18))

<a></a></a>

In [43]:
start_timer = time()

# Train classifier
imputer = preprocessing.Imputer()
scaler = preprocessing.MinMaxScaler()
clf = ensemble.AdaBoostClassifier(n_estimators=150) # n_estimators controls how many weak classifiers are fi

X_train_trans = imputer.fit_transform(X_train_shift)
X_train_trans = scaler.fit_transform(X_train_trans)
clf.fit(X_train_trans, Y_train_shift)

end_timer = time()

In [44]:
print "Time to train full ML pipline: %0.2f secs" % (end_timer - start_timer)

Time to train full ML pipline: 21.16 secs


In [45]:
Y_pred = clf.predict(X_train_trans)
print('Accuracy on train set = {:.2f}%'.format(metrics.accuracy_score(Y_train_shift, Y_pred) * 100))

Accuracy on train set = 56.20%


In [46]:
# Transform test data
X_test_trans = imputer.transform(X_test_shift)
X_test_trans = scaler.transform(X_test_trans)

In [47]:
# Predict!
Y_pred = clf.predict(X_test_trans)

Y_pred_prob = clf.predict_proba(X_test_trans)

In [48]:
print 'Predictions:', Y_pred
print 'Probabilities of class == 1:', Y_pred_prob[:, 1] * 100

Predictions: [-1. -1. -1. ...,  1.  1. -1.]
Probabilities of class == 1: [ 49.9863458   49.9550397   49.95409583 ...,  50.01426128  50.00151808
  49.9481115 ]


In [49]:
print('Accuracy on test set = {:.2f}%'.format(metrics.accuracy_score(Y_test_shift, Y_pred) * 100))
print('Log-loss = {:.5f}'.format(metrics.log_loss(Y_test_shift, Y_pred_prob)))

Accuracy on test set = 49.74%
Log-loss = 0.69315
